Construct a random 3×2 matrix.

Reduced SVD

Verify A=USV^T

### Understanding Singular Value Decomposition (SVD)

SVD is a powerful matrix factorization technique that decomposes a matrix `A` into three other matrices: `U`, `Σ` (Sigma), and `Vᵀ` (V-transpose).

$$ A = U \Sigma V^T $$

Where:
- `U` is an `m x m` orthogonal matrix (left singular vectors).
- `Σ` (Sigma) is an `m x n` diagonal matrix containing the singular values of `A` in descending order.
- `Vᵀ` is an `n x n` orthogonal matrix (right singular vectors, transposed).

The singular values are the square roots of the eigenvalues of `AᵀA` (or `AAᵀ`). The columns of `U` are the eigenvectors of `AAᵀ`, and the columns of `V` are the eigenvectors of `AᵀA`.

### Full SVD from Scratch

To implement full SVD from scratch, we will follow these steps:
1.  **Compute `V` and singular values:** Calculate `AᵀA`, then find its eigenvalues and eigenvectors. The eigenvectors form the columns of `V`, and the square roots of the eigenvalues are the singular values.
2.  **Compute `U`:** Calculate `AAᵀ`, then find its eigenvalues and eigenvectors. The eigenvectors form the columns of `U`.
3.  **Sort and Align:** Sort the eigenvalues/singular values in descending order and arrange the corresponding eigenvectors (`U` and `V`) accordingly. Crucially, align the signs of `U` and `V` to ensure `A = U Σ Vᵀ` holds true.

In [ ]:
import numpy as np

def svd_from_scratch_full(A):
    m, n = A.shape
    tolerance = 1e-9 # Tolerance for zero singular values

    # Step 1: Compute V (right singular vectors) and singular values from A.T @ A
    ATA = A.T @ A
    # np.linalg.eigh is used for symmetric matrices, which ATA is.
    eigen_vals_ATA, V_raw = np.linalg.eigh(ATA)

    # Sort eigenvalues in descending order and reorder eigenvectors accordingly
    # np.linalg.eigh returns eigenvalues in ascending order
    idx_V = eigen_vals_ATA.argsort()[::-1]
    eigen_vals_ATA = eigen_vals_ATA[idx_V]
    V = V_raw[:, idx_V]

    singular_values = np.sqrt(np.maximum(eigen_vals_ATA, 0))

    # Step 2: Compute U (left singular vectors) from A @ A.T
    AAT = A @ A.T
    # np.linalg.eigh is used for symmetric matrices, which AAT is.
    eigen_vals_AAT, U_raw = np.linalg.eigh(AAT)

    # Sort eigenvalues in descending order and reorder eigenvectors accordingly
    idx_U = eigen_vals_AAT.argsort()[::-1]
    eigen_vals_AAT = eigen_vals_AAT[idx_U]
    U = U_raw[:, idx_U]

    # Step 3: Align signs of U and V
    # The relationship A * v_i = s_i * u_i must hold. We align U based on this.
    for i in range(min(m, n)):
        if singular_values[i] > tolerance:
            # Calculate the candidate u_vector from A and V
            candidate_u = A @ V[:, i] / singular_values[i]
            # Compare its direction with the computed U[:, i]
            # If they are in opposite directions (dot product is negative), flip U[:, i]
            if np.dot(U[:, i], candidate_u) < 0:
                U[:, i] = -U[:, i]

    # Step 4: Construct Sigma matrix (m x n) with singular values on the diagonal
    Sigma_matrix = np.zeros((m, n))
    Sigma_matrix[:min(m, n), :min(m, n)] = np.diag(singular_values[:min(m, n)])

    return U, singular_values, V.T # Return V.T to match np.linalg.svd output convention

# Example Usage:
# Define a sample matrix A
A_full_example = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])

print("Original Matrix A (full example):\n", A_full_example)
U_fs, S_fs_diag, Vh_fs = svd_from_scratch_full(A_full_example)

print("\nU (from scratch, full):\n", U_fs)
print("\nSingular Values (from scratch, full):\n", S_fs_diag)
print("\nVh (from scratch, full):\n", Vh_fs)

# Verify reconstruction
Sigma_fs = np.zeros(A_full_example.shape)
Sigma_fs[:len(S_fs_diag), :len(S_fs_diag)] = np.diag(S_fs_diag)
Reconstructed_A_fs = U_fs @ Sigma_fs @ Vh_fs
print("\nReconstructed A (from scratch, full):\n", Reconstructed_A_fs)
print("\nMax absolute difference (A - Reconstructed A, full):", np.max(np.abs(A_full_example - Reconstructed_A_fs)))


Original Matrix A (full example):
 [[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]

U (from scratch, full):
 [[-0.14087668  0.82471435  0.47537679 -0.27206048]
 [-0.34394629  0.42626394 -0.83661763  0.00842237]
 [-0.54701591  0.02781353  0.24710489  0.79933671]
 [-0.75008553 -0.37063688  0.11413595 -0.5356986 ]]

Singular Values (from scratch, full):
 [25.46240744  1.29066168  0.        ]

Vh (from scratch, full):
 [[-0.50453315 -0.5745157  -0.64449826]
 [-0.76077568 -0.05714052  0.64649464]
 [ 0.40824829 -0.81649658  0.40824829]]

Reconstructed A (from scratch, full):
 [[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]]

Max absolute difference (A - Reconstructed A, full): 2.55351295663786e-14


### Reduced SVD from Scratch

Reduced SVD (also known as truncated SVD or thin SVD) is a variant where only the non-zero singular values and their corresponding singular vectors are kept. This is particularly useful for dimensionality reduction when the matrix is rank-deficient or when we only care about the principal components.

For a matrix `A` of shape `m x n` and rank `r`:
- `U_reduced` will be `m x r`.
- `Σ_reduced` will be `r x r` (a diagonal matrix).
- `Vᵀ_reduced` will be `r x n`.

We can leverage our `svd_from_scratch_full` function and simply truncate the matrices based on the number of non-zero singular values.

In [ ]:
import numpy as np

def svd_from_scratch_reduced(A):
    # Utilize the full SVD from scratch to get all components
    U_full, singular_values_full, Vh_full = svd_from_scratch_full(A)

    # Determine the effective rank by counting non-zero singular values
    tolerance = 1e-9 # Threshold for considering a singular value as non-zero
    rank = np.sum(singular_values_full > tolerance)

    # Truncate the matrices to keep only the components corresponding to non-zero singular values
    U_reduced = U_full[:, :rank]
    singular_values_reduced = singular_values_full[:rank]
    Vh_reduced = Vh_full[:rank, :]

    # Construct the reduced Sigma matrix (r x r diagonal matrix)
    Sigma_reduced_matrix = np.diag(singular_values_reduced)

    return U_reduced, singular_values_reduced, Vh_reduced

# Example Usage:
# Define a sample matrix A (same as full SVD example)
A_reduced_example = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])

print("Original Matrix A (reduced example):\n", A_reduced_example)
U_rs, S_rs_diag, Vh_rs = svd_from_scratch_reduced(A_reduced_example)

print("\nU (from scratch, reduced):\n", U_rs)
print("\nSingular Values (from scratch, reduced):\n", S_rs_diag)
print("\nVh (from scratch, reduced):\n", Vh_rs)

# Verify reconstruction (using the reduced matrices)
Sigma_rs = np.diag(S_rs_diag)
Reconstructed_A_rs = U_rs @ Sigma_rs @ Vh_rs
print("\nReconstructed A (from scratch, reduced):\n", Reconstructed_A_rs)
print("\nMax absolute difference (A - Reconstructed A, reduced):", np.max(np.abs(A_reduced_example - Reconstructed_A_rs)))


Original Matrix A (reduced example):
 [[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]

U (from scratch, reduced):
 [[-0.14087668  0.82471435]
 [-0.34394629  0.42626394]
 [-0.54701591  0.02781353]
 [-0.75008553 -0.37063688]]

Singular Values (from scratch, reduced):
 [25.46240744  1.29066168]

Vh (from scratch, reduced):
 [[-0.50453315 -0.5745157  -0.64449826]
 [-0.76077568 -0.05714052  0.64649464]]

Reconstructed A (from scratch, reduced):
 [[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]]

Max absolute difference (A - Reconstructed A, reduced): 2.55351295663786e-14


### Comparison with `np.linalg.svd`

Let's compare the results of our custom SVD implementations with NumPy's highly optimized `np.linalg.svd` function for verification. Note that `np.linalg.svd` might return `U` and `Vh` with arbitrary sign flips compared to our implementation, but the reconstruction `U @ diag(S) @ Vh` should still match the original matrix `A`.

In [ ]:
import numpy as np

# --- Test with a non-square matrix (e.g., 4x3) ---
print("\n--- Testing with a 4x3 matrix ---")
A_test = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])

print("Original Matrix A:\n", A_test)

# Full SVD from scratch
U_fs, S_fs_diag, Vh_fs = svd_from_scratch_full(A_test)
print("\nFull SVD from scratch results:")
print("Singular Values:\n", S_fs_diag)

# Full SVD using NumPy
U_np_full, S_np_full_diag, Vh_np_full = np.linalg.svd(A_test, full_matrices=True)
print("\nFull SVD using np.linalg.svd results:")
print("Singular Values:\n", S_np_full_diag)

# Reduced SVD from scratch
U_rs, S_rs_diag, Vh_rs = svd_from_scratch_reduced(A_test)
print("\nReduced SVD from scratch results:")
print("Singular Values:\n", S_rs_diag)

# Reduced SVD using NumPy
# np.linalg.svd(A, full_matrices=False) gives the reduced SVD
U_np_reduced, S_np_reduced_diag, Vh_np_reduced = np.linalg.svd(A_test, full_matrices=False)
print("\nReduced SVD using np.linalg.svd results:")
print("Singular Values:\n", S_np_reduced_diag)

print("\nComparison of Singular Values (Full SVD):")
print("From Scratch:", S_fs_diag)
print("NumPy (full): ", S_np_full_diag)
print("Are singular values from scratch and numpy full SVD close?: ", np.allclose(S_fs_diag, S_np_full_diag[:len(S_fs_diag)]))

print("\nComparison of Singular Values (Reduced SVD):")
print("From Scratch:", S_rs_diag)
print("NumPy (reduced): ", S_np_reduced_diag)
print("Are singular values from scratch and numpy reduced SVD close?: ", np.allclose(S_rs_diag, S_np_reduced_diag[:len(S_rs_diag)]))

# --- Test with a rank-deficient matrix (e.g., 3x3) ---
print("\n--- Testing with a rank-deficient 3x3 matrix ---")
# This matrix has rank 1
A_rank_deficient = np.array([
    [1, 1, 1],
    [2, 2, 2],
    [3, 3, 3]
])

print("Original Rank-Deficient Matrix A:\n", A_rank_deficient)

# Full SVD from scratch
U_fs_rd, S_fs_rd_diag, Vh_fs_rd = svd_from_scratch_full(A_rank_deficient)
print("\nFull SVD from scratch (rank deficient):")
print("Singular Values:\n", S_fs_rd_diag)

# Full SVD using NumPy
U_np_full_rd, S_np_full_rd_diag, Vh_np_full_rd = np.linalg.svd(A_rank_deficient, full_matrices=True)
print("\nFull SVD using np.linalg.svd (rank deficient):")
print("Singular Values:\n", S_np_full_rd_diag)

# Reduced SVD from scratch
U_rs_rd, S_rs_rd_diag, Vh_rs_rd = svd_from_scratch_reduced(A_rank_deficient)
print("\nReduced SVD from scratch (rank deficient):")
print("Singular Values:\n", S_rs_rd_diag)

# Reduced SVD using NumPy
U_np_reduced_rd, S_np_reduced_rd_diag, Vh_np_reduced_rd = np.linalg.svd(A_rank_deficient, full_matrices=False)
print("\nReduced SVD using np.linalg.svd (rank deficient):")
print("Singular Values:\n", S_np_reduced_rd_diag)

print("\nComparison of Singular Values (Full SVD, rank deficient):")
print("From Scratch:", S_fs_rd_diag)
print("NumPy (full): ", S_np_full_rd_diag)
print("Are singular values from scratch and numpy full SVD close?: ", np.allclose(S_fs_rd_diag, S_np_full_rd_diag[:len(S_fs_rd_diag)]))

print("\nComparison of Singular Values (Reduced SVD, rank deficient):")
print("From Scratch:", S_rs_rd_diag)
print("NumPy (reduced): ", S_np_reduced_rd_diag)
print("Are singular values from scratch and numpy reduced SVD close?: ", np.allclose(S_rs_rd_diag, S_np_reduced_rd_diag[:len(S_rs_rd_diag)]))



--- Testing with a 4x3 matrix ---
Original Matrix A:
 [[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]

Full SVD from scratch results:
Singular Values:
 [25.46240744  1.29066168  0.        ]

Full SVD using np.linalg.svd results:
Singular Values:
 [2.54624074e+01 1.29066168e+00 2.40694596e-15]

Reduced SVD from scratch results:
Singular Values:
 [25.46240744  1.29066168]

Reduced SVD using np.linalg.svd results:
Singular Values:
 [2.54624074e+01 1.29066168e+00 2.40694596e-15]

Comparison of Singular Values (Full SVD):
From Scratch: [25.46240744  1.29066168  0.        ]
NumPy (full):  [2.54624074e+01 1.29066168e+00 2.40694596e-15]
Are singular values from scratch and numpy full SVD close?:  True

Comparison of Singular Values (Reduced SVD):
From Scratch: [25.46240744  1.29066168]
NumPy (reduced):  [2.54624074e+01 1.29066168e+00 2.40694596e-15]
Are singular values from scratch and numpy reduced SVD close?:  True

--- Testing with a rank-deficient 3x3 matrix ---
Original Rank-Deficient M

Rank deficient case